# 一、针对序列级和词元级应用微调BERT

本章的前几节中，我们为自然语言处理应用设计了不同的模型。例如基于循环神经网络、卷积神经网络、注意力和多层感知机。这些模型在有空间或时间限制的情况下是有帮助的，但是，为每一个自然语言处理任务精心设计一个特定的模型实际上是不可行的。在 14.8节中，我们介绍了一个名为Bert的预训练模型，该模型可以对广泛的自然语言处理任务进行最少的架构更改。一方面，在提出时，Bert给i尽量各种自然语言处理任务的水平。另一方面，正如在 14.10节中指出的那样，原始BERT模型的两个版本分别带有1.1亿和3.4亿个参数。因此，当有足够的计算资源时，我们可以考虑为下游自然语言处理应用微调BERT。

下面，我们将自然语言处理应用的子集概括为序列级和词元级。在序列层次上，介绍了在单文本分类任务和文本对分类（或回归）任务中，如何将文本输入的BERT表示转换为输出标签。在词元级别，我们将简要介绍新的应用，如文本标注和问答，并说明BERT如何表示它们的输入并转换为输出标签。在微调期间，不同应用之间的BERT所需的“最小架构更改”是额外的全连接层。在下游应用的监督学习期间，额外层的参数是从零开始学习的，而预训练BERT模型中的所有参数都是微调的。

## 1.1 单文本分类

单文本分类将单个文本序列作为输入，并输出其分类结果。 除了我们在这一章中探讨的情感分析之外，语言可接受性语料库（Corpus of Linguistic Acceptability，COLA）也是一个单文本分类的数据集，它的要求判断给定的句子在语法上是否可以接受。 (Warstadt et al., 2019)。例如，“I should study.”是可以接受的，但是“I should studying.”不是可以接受的。

![](https://files.mdnice.com/user/23981/c2510977-8f07-47a7-87b6-cb4e484b3573.png)

14.8节描述了BERT的输入表示。BERT输入序列明确地表示单个文本和文本对，其中特殊分类标记“<cls>”用于序列分类，而特殊分类标记“<sep>”标记单个文本的结束或分隔成对文本。如 图15.6.1所示，在单文本分类应用中，特殊分类标记“<cls>”的BERT表示对整个输入文本序列的信息进行编码。作为输入单个文本的表示，它将被送入到由全连接（稠密）层组成的小多层感知机中，以输出所有离散标签值的分布。

## 1.2 文本对分类或回归

在本章中，我们还研究了自然语言推断。它属于文本对分类，这是一种对文本进行分类的应用类型。

以一对文本作为输入但输出连续值，语义文本相似度是一个流行的“文本对回归”任务。 这项任务评估句子的语义相似度。例如，在语义文本相似度基准数据集（Semantic Textual Similarity Benchmark）中，句子对的相似度得分是从0（无语义重叠）到5（语义等价）的分数区间 (Cer et al., 2017)。我们的目标是预测这些分数。来自语义文本相似性基准数据集的样本包括（句子1，句子2，相似性得分）：

* “A plane is taking off.”（“一架飞机正在起飞。”），”An air plane is taking off.”（“一架飞机正在起飞。”），5.000分;
* “A woman is eating something.”（“一个女人在吃东西。”），”A woman is eating meat.”（“一个女人在吃肉。”），3.000分;
* “A woman is dancing.”（一个女人在跳舞。），”A man is talking.”（“一个人在说话。”），0.000分。

![](https://files.mdnice.com/user/23981/5c1c0ce3-72fe-49de-b3b0-ecbf1910a883.png)

与 图15.6.1中的单文本分类相比， 图15.6.2中的文本对分类的BERT微调在输入表示上有所不同。对于文本对回归任务（如语义文本相似性），可以应用细微的更改，例如输出连续的标签值和使用均方损失：它们在回归中很常见。

## 1.3 文本标注

现在让我们考虑词元级任务，比如文本标注（text tagging），其中每个词元都被分配了一个标签。在文本标注任务中，词性标注为每个单词分配词性标记（例如，形容词和限定词）。 根据单词在句子中的作用。如，在Penn树库II标注集中，句子“John Smith‘s car is new”应该被标记为“NNP（名词，专有单数）NNP POS（所有格结尾）NN（名词，单数或质量）VB（动词，基本形式）JJ（形容词）”。

![](https://files.mdnice.com/user/23981/a9ad61d5-6afb-4c70-b150-1abd33d0754c.png)

图15.6.3中说明了文本标记应用的BERT微调。与 图15.6.1相比，唯一的区别在于，在文本标注中，输入文本的每个词元的BERT表示被送到相同的额外全连接层中，以输出词元的标签，例如词性标签。

## 1.4 问答

作为另一个词元级应用，问答反映阅读理解能力。 例如，斯坦福问答数据集（Stanford Question Answering Dataset，SQuAD v1.1）由阅读段落和问题组成，其中每个问题的答案只是段落中的一段文本（文本片段） (Rajpurkar et al., 2016)。举个例子，考虑一段话：“Some experts report that a mask’s efficacy is inconclusive.However,mask makers insist that their products,such as N95 respirator masks,can guard against the virus.”（“一些专家报告说面罩的功效是不确定的。然而，口罩制造商坚持他们的产品，如N95口罩，可以预防病毒。”）还有一个问题“Who say that N95 respirator masks can guard against the virus?”（“谁说N95口罩可以预防病毒？”）。答案应该是文章中的文本片段“mask makers”（“口罩制造商”）。因此，SQuAD v1.1的目标是在给定问题和段落的情况下预测段落中文本片段的开始和结束。

![](https://files.mdnice.com/user/23981/30650568-4e31-4a1c-af61-f179a8eca43f.png)

为了微调BERT进行问答，在BERT的输入中，将问题和段落分别作为第一个和第二个文本序列。为了预测文本片段开始的位置，相同的额外的全连接层将把来自位置2的任何词元的BERT表示转换成标量分数si。文章中所有词元的分数还通过softmax转换成概率分布，从而为文章中的每个词元位置份配作为文本片段开始的概率p。预测文本片段的结束与上面相同，只是其额外的全连接层中的参数与用于预测开始位置的参数无关。当预测结束时，位置的词元由相同的全连接层变换成标量分数eio图15.6.4描述了用于问答的微调BERT。
对于问答，监督学习的训练目标就像最大化真实值的开始和结束位置的对数似然一样简单。当预测片段时，我们可以计算从位置到位置3的有效片段的分数si+ej（2<j），并输出分数最高的跨度。

# 二、自然语言推断：微调BERT

在本章的前面几节中，我们已经为SNLI数据集（ 15.4节）上的自然语言推断任务设计了一个基于注意力的结构（ 15.5节）。现在，我们通过微调BERT来重新审视这项任务。正如在 15.6节中讨论的那样，自然语言推断是一个序列级别的文本对分类问题，而微调BERT只需要一个额外的基于多层感知机的架构，如 图15.7.1中所示。

![](https://files.mdnice.com/user/23981/68549b20-22fa-4c73-9547-c0252475ef7b.png)

本节将下载一个预训练好的小版本的BERT，然后对其进行微调，以便在SNLI数据集上进行自然语言推断。

In [1]:
import json
import multiprocessing
import os
import torch
from torch import nn
from d2l import torch as d2l
import numpy as np

## 2.1 加载预训练好的Bert



我们已经在 14.9节和 14.10节WikiText-2数据集上预训练BERT（请注意，原始的BERT模型是在更大的语料库上预训练的）。正如在 14.10节中所讨论的，原始的BERT模型有数以亿计的参数。在下面，我们提供了两个版本的预训练的BERT：“bert.base”与原始的BERT基础模型一样大，需要大量的计算资源才能进行微调，而“bert.small”是一个小版本，以便于演示。

In [2]:
d2l.DATA_HUB['bert.base'] = (d2l.DATA_URL + 'bert.base.torch.zip',
                             '225d66f04cae318b841a13d32af3acc165f253ac')
d2l.DATA_HUB['bert.small'] = (d2l.DATA_URL + 'bert.small.torch.zip',
                              'c72329e68a732bef0452e4b96a1c341c8910f81f')

两个预训练好的BERT模型都包含一个定义词表的“vocab.json”文件和一个预训练参数的“pretrained.params”文件。我们实现了以下load_pretrained_model函数来加载预先训练好的BERT参数。

In [3]:
def load_pretrained_model(pretrained_model, num_hiddens, ffn_num_hiddens,
                          num_heads, num_layers, dropout, max_len, devices):
    data_dir = d2l.download_extract(pretrained_model)
    # 定义空词表以加载预定义词表
    vocab = d2l.Vocab()
    vocab.idx_to_token = json.load(open(os.path.join(data_dir,
        'vocab.json')))
    vocab.token_to_idx = {token: idx for idx, token in enumerate(
        vocab.idx_to_token)}
    bert = d2l.BERTModel(len(vocab), num_hiddens, norm_shape=[256],
                         ffn_num_input=256, ffn_num_hiddens=ffn_num_hiddens,
                         num_heads=4, num_layers=2, dropout=0.2,
                         max_len=max_len, key_size=256, query_size=256,
                         value_size=256, hid_in_features=256,
                         mlm_in_features=256, nsp_in_features=256)
    # 加载预训练BERT参数
    bert.load_state_dict(torch.load(os.path.join(data_dir, 'pretrained.params')))
    return bert, vocab

为了便于在大多数机器上演示，我们将在本节中加载和微调经过预训练BERT的小版本（“bert.small”）。在练习中，我们将展示如何微调大得多的“bert.base”以显著提高测试精度。

In [4]:
devices = d2l.try_all_gpus()
bert, vocab = load_pretrained_model(
    'bert.small', num_hiddens=256, ffn_num_hiddens=512, num_heads=4,
    num_layers=2, dropout=0.1, max_len=512, devices=devices)

## 2.2 微调BERT的数据集

对于SNLI数据集的下游任务自然语言推断，我们定义了一个定制的数据集类SNLIBERTDataset。在每个样本中，前提和假设形成一对文本序列，并被打包成一个BERT输入序列，如 图15.6.2所示。回想 14.8.4节，片段索引用于区分BERT输入序列中的前提和假设。利用预定义的BERT输入序列的最大长度（max_len），持续移除输入文本对中较长文本的最后一个标记，直到满足max_len。为了加速生成用于微调BERT的SNLI数据集，我们使用4个工作进程并行生成训练或测试样本。

In [5]:
class SNLIBERTDataset(torch.utils.data.Dataset):
    def __init__(self,dataset,max_len,vocab=None):
        """
        :param dataset: 数据集
        :param max_len: 序列最大长度
        :param vocab: 词表
        """
        # 将前提和假设的文本数据进行分词，分词后的结果存储在all_premise_hypothesis_tokens中
        all_premise_hypothesis_tokens=[
            # p_tokens和h_tokens分别指的是前提和假设的向量表示
            [p_tokens,h_tokens] for p_tokens,h_tokens in zip(
                *[d2l.tokenize([s.lower() for s in sentences])
                  for sentences in dataset[:2]]
            )
        ]
        self.labels=torch.tensor(dataset[2])    #标签
        self.vocab=vocab        # 词表
        self.max_len=max_len    # 最大长度
        # 对所有的前提和假设进行预处理
        (self.all_token_ids,self.all_segments,self.valid_len)=self._preprocess(all_premise_hypothesis_tokens)
        print('read ' + str(len(self.all_token_ids)) + ' examples')

    def _preprocess(self,all_premise_hypothesis_tokens):
        pool=multiprocessing.Pool(4)    #使用4个进程
        # map()方法接收一个函数（也就是self._mp_worker）和一个可迭代对象（也就是all_premise_hypothesis_tokens），将可迭代对象中的每个元素依次传入函数进行处理，并返回一个列表，其中包含了所有结果。
        # 使用进程池中的进程对所有的前提和假设进行处理，将处理后的结果保存在 out 列表中。
        out=pool.map(self._mp_worker,all_premise_hypothesis_tokens)
        all_token_ids=[token_ids for token_ids,segments,valid_len in out]
        all_segments=[segments for token_ids,segments,valid_len in out]
        valid_lens=[valid_len for token_ids,segments,valid_len in out]
        return (np.array(all_token_ids, dtype='int32'),
                np.array(all_segments, dtype='int32'),
                np.array(valid_lens))

    def _mp_worker(self,premise_hypothesis_tokens):
        """
        多进程处理函数，接收一个包含前提和假设的文本数据的元组 premise_hypothesis_tokens。
        :param premise_hypothesis_tokens: 前提和假设的文本数据的元组
        :return:
        """
        p_tokens,h_tokens=premise_hypothesis_tokens
        # 将前提和假设的文本进行截断处理，得到截断后的词元列表
        self._truncate_pair_of_tokens(p_tokens,h_tokens)
        tokens,segments=d2l.get_tokens_and_segments(p_tokens,h_tokens)
        # 根据截断后的词元列表获取词元的索引，并进行填充操作
        token_ids=self.vocab[tokens]+[self.vocab['<pad>']]*(self.max_len-len(tokens))
        segments=segments+[0]*(self.max_len-len(segments))
        valid_len=len(tokens)
        return token_ids,segments,valid_len

    def _truncate_pair_of_tokens(self,p_tokens,h_tokens):
        # 对前提和假设的词元进行截断处理，使其总长度不超过设定的最大长度
        # 为Bert输入中的'<cls>'、'<sep>'和‘<sep>'词元保留位置
        while len(p_tokens) + len(h_tokens)>self.max_len-3:
            if len(p_tokens)>len(h_tokens):
                p_tokens.pop()  #移除列表中的最后一个元素，并返回被移除的元素。以实现截断操作，以确保前提文本的词元数量不超过指定的最大长度
            else:
                h_tokens.pop()

    def __getitem__(self, idx):
        # 获取数据集中指定索引的样本
        return (self.all_token_ids[idx], self.all_segments[idx],
                self.valid_lens[idx]), self.labels[idx]

    def __len__(self):
        return len(self.all_token_ids)

下载完SNLI数据集后，我们通过实例化SNLIBERTDataset类来生成训练和测试样本。这些样本将在自然语言推断的训练和测试期间进行小批量读取。

In [ ]:
# 如果出现显存不足错误，请减少batch_size。在原始的Bert模型中，max_len=512
batch_size,max_len,num_workers=512,128,d2l.get_dataloader_workers()
data_dir="..\\data\\snli_1.0"
train_set=SNLIBERTDataset(d2l.read_snli(data_dir,True),max_len,vocab)
test_set=SNLIBERTDataset(d2l.read_snli(data_dir,False),max_len,vocab)
train_iter = torch.utils.data.DataLoader(train_set, batch_size, shuffle=True,
                                         num_workers=num_workers)
test_iter = torch.utils.data.DataLoader(test_set, batch_size,
                                        num_workers=num_workers)

Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\self_download\miniconda3\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\self_download\miniconda3\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\self_download\miniconda3\lib\multiprocessing\pool.py", line 513, in _handle_workers
    cls._maintain_pool(ctx, Process, processes, pool, inqueue,
  File "C:\self_download\miniconda3\lib\multiprocessing\pool.py", line 337, in _maintain_pool
    Pool._repopulate_pool_static(ctx, Process, processes, pool,
  File "C:\self_download\miniconda3\lib\multiprocessing\pool.py", line 326, in _repopulate_pool_static
    w.start()
  File "C:\self_download\miniconda3\lib\multiprocessing\process.py", line 121, in start
    self._popen = self._Popen(self)
  File "C:\self_download\miniconda3\lib\multiprocessing\context.py", line 327, in _Popen
    return Popen(process_obj)
  File "C:\self_download\miniconda3\lib\m

## 2.3 微调BERT

如 图15.6.2所示，用于自然语言推断的微调BERT只需要一个额外的多层感知机，该多层感知机由两个全连接层组成（请参见下面BERTClassifier类中的self.hidden和self.output）。这个多层感知机将特殊的“<cls>”词元的BERT表示进行了转换，该词元同时编码前提和假设的信息为自然语言推断的三个输出：蕴涵、矛盾和中性。

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,bert):
        super(BERTClassifier, self).__init__()
        self.encoder=bert.encoder
        self.hidden=bert.hidden
        self.output=nn.Linear(256,3)

    def forward(self,inputs):
        tokens_X,segments_X,valid_lens_x=inputs
        encoded_X=self.encoder(tokens_X,segments_X,valid_lens_x)
        return self.output(self.hidden(encoded_X[:,0,:]))

在下文中，预训练的BERT模型bert被送到用于下游应用的BERTClassifier实例net中。在BERT微调的常见实现中，只有额外的多层感知机（net.output）的输出层的参数将从零开始学习。预训练BERT编码器（net.encoder）和额外的多层感知机的隐藏层（net.hidden）的所有参数都将进行微调。

In [ ]:
net = BERTClassifier(bert)

回想一下，在 14.8节中，MaskLM类和NextSentencePred类在其使用的多层感知机中都有一些参数。这些参数是预训练BERT模型bert中参数的一部分，因此是net中的参数的一部分。然而，这些参数仅用于计算预训练过程中的遮蔽语言模型损失和下一句预测损失。这两个损失函数与微调下游应用无关，因此当BERT微调时，MaskLM和NextSentencePred中采用的多层感知机的参数不会更新（陈旧的，staled）。

为了允许具有陈旧梯度的参数，标志ignore_stale_grad=True在step函数d2l.train_batch_ch13中被设置。我们通过该函数使用SNLI的训练集（train_iter）和测试集（test_iter）对net模型进行训练和评估。由于计算资源有限，训练和测试精度可以进一步提高：我们把对它的讨论留在练习中。

In [ ]:
lr,num_epochs=1e-4,5
trainer=torch.optim.Adam(net.parameters(),lr=lr)
loss=nn.CrossEntropyLoss(reduction='none')
d2l.train_ch13(net,train_iter,test_iter,loss,trainer,num_epochs,devices)